In [415]:
#Imports
import numpy as np
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import matplotlib.pyplot as plt

In [416]:
#--------------CONSTANTES ROBO Pioneer-----------------
ROBOTNAME = 'Pioneer_p3dx'
robotLeftMotorName = '/' + ROBOTNAME + '_leftMotor'
robotRightMotorName = "/" + ROBOTNAME + '_rightMotor'

L = 0.331
r = 0.0975

maxv = 1.0 #LIMITADORES DA VELOCIDADE LINEAR
maxw = np.deg2rad(45) #LIMITADORES DA VELOCIDADE ANGULAR
max_sensor_range=5 
rho = np.inf
random_force = 0.1

In [417]:
#faz conexão com a cena
try:
    client = RemoteAPIClient()
    sim = client.require('sim')
except:
    print('Error connection')

**Plots e Leituras do sensor**

In [418]:
# --------------- Funções auxiliares - Tranformação de Matriz, plot Sistema de Coordenadas e Muda de posição Robô -----------------
def get_rotation_matrix(thetaZ):
    return np.array([[  np.cos(thetaZ), -np.sin(thetaZ), 0 ],
                      [ np.sin(thetaZ),  np.cos(thetaZ), 0 ],
                      [ 0            ,  0            , 1 ]])

def degrees_to_radians(angleDegrees):
    return np.deg2rad(angleDegrees)

def plot_coordinate_system(Porg, Rotation, c=['r', 'g']):
    #Porg - Origem, Roation - Rotação do meu sistema de coordenadas (matriz de rotação) de um referencial global para o local
    axis_size = 1    
    axes = axis_size*Rotation
    
    x_axis = np.array(axes[0:2,0])
    y_axis = np.array(axes[0:2,1])
        
    # X
    plt.quiver(*Porg[:2], *x_axis, color=c[0], angles='xy', scale_units='xy', scale=1)
    
    # Y
    plt.quiver(*Porg[:2], *y_axis, color=c[1], angles='xy', scale_units='xy', scale=1)

In [419]:
# ------------------ Funções auxiliares - Lê Dados do sensor, Pré-processamento dos dados do laser e Desenha os pontos lidos pelo sensor ----------------
def readSensorData(range_data_signal_id="hokuyo_range_data", 
                    angle_data_signal_id="hokuyo_angle_data"):
    
    signalName = sim.waitForSignal(range_data_signal_id)
    
    string_range_data = sim.getStringSignal(range_data_signal_id)

    string_angle_data = sim.getStringSignal(angle_data_signal_id)

    # verifique se ambos os dados foram obtidos corretamente
    if string_range_data != None and string_angle_data != None:
        # descompacte dados de mensagens de alcance e sensor
        raw_range_data = sim.unpackFloatTable(string_range_data)
        raw_angle_data = sim.unpackFloatTable(string_angle_data)

        return raw_range_data, raw_angle_data

    # retornar nenhum caso nada tenha sido obtido do simulador
    return None


def draw_laser_data(laser_data, transformationMatrix, origin, graph_title):
    
    fig = plt.figure(figsize=(6,6), dpi=100) #cria figura
    plt.title(graph_title)
    ax = fig.add_subplot(111, aspect='equal') #adiciona um subplot a figura
    max_sensor_range=5
        
    for i in range(len(laser_data)):
        ang, dist = laser_data[i] #pega os valores de angulo e distância

        if (max_sensor_range - dist) > 0.1:
            x = dist * np.cos(ang) #meu x
            y = dist * np.sin(ang) #meu y
            point = np.array([x,y,0,1])

            if len(transformationMatrix) != 0:
                point = transformationMatrix @ point

            c = 'r'
            if ang < 0:    
                c = 'b'
            ax.plot(point[0], point[1], 'o', color=c) #plota o ponto

    ax.plot(origin[0], origin[1], 'k>', markersize=10) #A origem é a posição atual do robô
        
    ax.grid() #adiciona uma grade ao gráfico
    ax.set_xlim([-10, 1]) #define os limites do eixo x
    ax.set_ylim([-10, 1]) #define os limites eixo y


In [420]:
# def get_rotation_matrix_laser_world():
#     objectHandle_RP = sim.getObject("/" + "Pioneer_p3dx")

#     objectHandle_Laser = sim.getObject("/" + "fastHokuyo")
#     eulerAngles_Laser = sim.getObjectOrientation(objectHandle_Laser, 
#                                                 objectHandle_RP)

#     #Matriz de Rotação Laser -> Robô
#     rotationMatrix_Laser = get_rotation_matrix(eulerAngles_Laser[2])

#     #Matriz de Rotação Robô -> mundo
#     eulerAngles_RP = sim.getObjectOrientation(objectHandle_RP, 
#                                                 sim.handle_world)
#     rotationMatrix_RP = get_rotation_matrix(eulerAngles_RP[2])
#     #Matriz de Transformação Laser -> Mundo
#     return rotationMatrix_Laser @ rotationMatrix_RP

def get_rotation_matrix_laser_world():
    objectHandle_RP = sim.getObject("/" + "Pioneer_p3dx")
    origin_RP = np.array(sim.getObjectPosition(objectHandle_RP, 
                                        sim.handle_world))

    objectHandle_Laser = sim.getObject("/" + "fastHokuyo")
    origin_Laser = np.array(sim.getObjectPosition(objectHandle_Laser, 
                                        sim.handle_world))
    p_Laserorg = origin_Laser - origin_RP
    eulerAngles_Laser = sim.getObjectOrientation(objectHandle_Laser, 
                                                objectHandle_RP)

    aux = np.array([0, 0, 0, 1])
    rotationMatrix_Laser = get_rotation_matrix(eulerAngles_Laser[2])
    transformationMatrix_Laser = np.column_stack((rotationMatrix_Laser, p_Laserorg))
    transformationMatrix_Laser = np.row_stack((transformationMatrix_Laser, aux))

    # print(transformationMatrix_Laser)

    #Matriz de Transformação Robô -> mundo
    origin_World = np.array([0,0,0])
    p_RPorg = origin_RP - origin_World
    eulerAngles_RP = sim.getObjectOrientation(objectHandle_RP, 
                                                sim.handle_world)
    rotationMatrix_RP = get_rotation_matrix(eulerAngles_RP[2])
    transformationMatrix_World = np.column_stack((rotationMatrix_RP, p_RPorg))
    transformationMatrix_World = np.row_stack((transformationMatrix_World, aux))
    #Matriz de Transformação Laser -> Mundo
    return transformationMatrix_Laser @ transformationMatrix_World

**CALCULO DAS FORÇAS**

**1-Força Atração**

In [421]:
def att_force(q, goal, k=2):
    attraction_force = k*(goal - q[0:2])
    return attraction_force


**2-Força Repulsão**

In [422]:
#Realiza o calculo da força de repulsão
#krep -> Constante de ganho
#pi(q) -> distância do objeto para o feixe
#po -> constante que representa a qual o obstáculo gera uma força de repulsão
#y -> constante de valor 2
#q -> Posição atual do robo
#obs -> coordenada x,y do ponto que o feixe detectou o obstáculo
def rep_force(q, obs, R=3, krep=.1):
    v = q[0:2] - obs[0:2]
    d = np.linalg.norm(v) - obs[2]
    rep = (1/d**2) * ((1/d)-(1/R)) * (v/d)
    
    invalid = np.squeeze(d>R)
    rep[invalid, :] = 0
    return np.array(krep)*np.array(rep)



In [423]:
sim.startSimulation()

#---------ROBOS HANDLES-------------
robotHandle = sim.getObject('/' + ROBOTNAME)  
robotLeftMotorHandle = sim.getObject(robotLeftMotorName)
robotRightMotorHandle = sim.getObject(robotRightMotorName)

qgoal = np.array([-3.500, +3.000])

Frep_total = np.array([0.0, 0.0])
t=0
while rho > .05:
# while t<2:
    robotPos = sim.getObjectPosition(robotHandle) #Posição do robô no mundo
    robotOri = sim.getObjectOrientation(robotHandle) #Orientação do robô
    robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]]) #Configuração do robô

    #Obter o meu obstáculo a partir da leitura do laser
    raw_range_data, raw_angle_data = readSensorData()
    laser_data = np.array([raw_angle_data, raw_range_data]).T

    #Transformação dos dados do meu laser para o mundo
    transformationMatrix_LaserWorld = get_rotation_matrix_laser_world()
    #Desenhando a visualização do meu robo
    # draw_laser_data(laser_data, rotationMatrix_LaserWorld, np.array([robotConfig[0],robotConfig[1]]), "Visualização dos obstáculos")
    t+=1

    #Pegando os dados do meu laser
    for i in range(len(laser_data)):
        ang, dist = laser_data[i] #pega os valores de angulo e distância

        if (max_sensor_range - dist) > 0.1:
            x = dist * np.cos(ang) #meu x
            y = dist * np.sin(ang) #meu y
            point = np.array([x,y,0,1])

            if len(transformationMatrix_LaserWorld) != 0:
                point = transformationMatrix_LaserWorld @ point
            Frep = np.array(rep_force(robotPos, point))
        else:
            Frep = 0
        Frep_total = np.array(Frep_total) + np.array(Frep)
    
    # # Calcular as forças de atração e repulsão
    Fatt = att_force(robotPos, qgoal)
    random_force_vector = random_force * np.random.rand(2)
    Ft = Fatt + Frep + random_force_vector
    Ft_x = Ft[0]
    Ft_y = Ft[1]

    # # Apenas para interromper o loop
    rho = np.sqrt(Ft_x**2 + Ft_y**2)
    
    # # Formula De Luca e Oriolo para calcular a velocidade linear e angular.
    kr = 1
    kt = 2
    v = kr*(Ft_x*np.cos(robotConfig[2]) + Ft_y*np.sin(robotConfig[2]))
    w = kt*(np.arctan2(Ft_y,Ft_x) - robotConfig[2])
            
    # # Limit v,w to +/- max
    v = max(min(v, maxv), -maxv)
    w = max(min(w, maxw), -maxw)        
    
    vr = ((1.0*v) + (w*L))/(2.0*r)
    vl = ((1.0*v) - (w*L))/(2.0*r)
    sim.setJointTargetVelocity(robotRightMotorHandle, vr)
    sim.setJointTargetVelocity(robotLeftMotorHandle, vl)


sim.setJointTargetVelocity(robotRightMotorHandle, 0)
sim.setJointTargetVelocity(robotLeftMotorHandle, 0)

sim.stopSimulation()


KeyboardInterrupt: 